# KOMET Project Evaluation Report

![KOMET Project](https://projects.tib.eu/fileadmin/templates/komet/tib_projects_komet_1150.png)

## About KOMET

**KOMET** (Kollaborative Anreicherung der Metadatenallmende zur Förderung eines diversen Open-Access-Ökosystems) enhances metadata processes for independent, scholar-led Open Access journals. The project develops plugins and tools for Open Journal Systems (OJS) to improve article discoverability through:

- **PID Plugin**: Integrating validated persistent identifiers (ROR, IGSN, PIDINST, ConfIDent)
- **Citations Plugin**: Structured citation metadata capture and publication to OpenCitations
- **Geo Plugin**: Geospatial and temporal metadata for location-based article discovery

**Funding**: BMBF (German Federal Ministry of Education and Research)  
**Duration**: September 2023 - August 2025  
**Partners**: TIB Hannover (coordinator), TU Dresden

### Project Links

| Resource | Link |
|----------|------|
| Project Website | https://projects.tib.eu/komet |
| Predecessor (OPTIMETA) | https://projects.tib.eu/optimeta |
| Mastodon | https://openbiblio.social/@komet |

### Code Repositories

| Repository | Description |
|------------|-------------|
| [optimetaCitations](https://github.com/TIBHannover/optimetaCitations) | OJS Citations Plugin |
| [optimetaGeo](https://github.com/TIBHannover/optimetaGeo) | OJS Geospatial Metadata Plugin |
| [optimeta-plugin-shared](https://github.com/TIBHannover/optimeta-plugin-shared) | Shared plugin library |
| [OpenCitations Crowdsourcing](https://github.com/opencitations/crowdsourcing) | Citation contribution platform |

### Demos & Tools

| Tool | Link |
|------|------|
| OPTIMAP | https://optimap.science/ |
| Scholia (Wikidata visualization) | https://scholia.toolforge.org/ |

---

## This Notebook: AP 4.3 Evaluation

This notebook implements **Work Package 4.3: Evaluation** from the KOMET project proposal, tracking contributions to the open metadata commons.

### Data Sources & Platform Priority

**Primary target platform**: OpenCitations (via GitHub crowdsourcing)  
**Secondary/monitoring**: Wikidata (limited reach, documented below)

| Source | Purpose | Priority |
|--------|---------|----------|
| OpenCitations | Citation metadata contributions | Primary |
| Wikidata | Scholarly graph baseline & P2860 citations | Monitoring |
| OpenAIRE | Future: citation data exploration | Planned |

In [1]:
# Setup and imports
import komet_helpers as kh
from datetime import datetime
import json
import os

# Load or create timeline for tracking metrics over time
timeline = kh.load_timeline()
print(f"Report generated: {kh.format_timestamp()}")
print(f"Timeline created: {timeline.get('created', 'N/A')}")
print(f"Timeline last updated: {timeline.get('last_updated', 'Never')}")
print(f"Total observations: {len(timeline.get('observations', []))}")

Report generated: 2026-01-22T11:30:11.762676Z
Timeline created: 2026-01-22T11:30:11.762572Z
Timeline last updated: None
Total observations: 0


---

## 1. Wikidata Analysis

### 1.1 P1343 (Described by Source) - Limited Reach Analysis

The original KOMET proposal mentioned using **P1343 ("described by source")** to mark contributions from project plugins. However, our analysis reveals this property has **very limited usage** for scholarly articles.

**Important Context**:
- P1343 is primarily used to indicate items described in encyclopedias/reference works
- It is **NOT** the standard property for tracking citations in Wikidata
- The primary citation property is **P2860 ("cites work")** with ~3.4M relationships
- **Wikidata is no longer the primary target platform** - OpenCitations is the focus

In [2]:
# Count scholarly articles with P1343
print("Querying Wikidata for P1343 usage on scholarly articles...")
p1343_count = kh.count_p1343_scholarly_articles()

if p1343_count is not None:
    print(f"\nScholarly articles with P1343 (described by source): {p1343_count}")
    print("\n" + "="*60)
    print("LIMITATION: This is an extremely low number compared to")
    print("~37 million scholarly articles in Wikidata.")
    print("P1343 is NOT suitable for tracking KOMET contributions.")
    print("="*60)
    
    # Log to timeline
    added = kh.add_observation(
        timeline, 
        "wikidata_p1343_scholarly_count", 
        p1343_count,
        "wikidata",
        "P1343 rarely used for scholarly articles - not primary tracking method"
    )
    print(f"\nTimeline: {'New observation added' if added else 'Value unchanged, timestamp updated'}")
else:
    print("Query failed or timed out")

Querying Wikidata for P1343 usage on scholarly articles...



Scholarly articles with P1343 (described by source): 14

LIMITATION: This is an extremely low number compared to
~37 million scholarly articles in Wikidata.
P1343 is NOT suitable for tracking KOMET contributions.

Timeline: New observation added


In [3]:
# Search for any existing KOMET/OPTIMETA references
print("Searching for KOMET/OPTIMETA provenance markers in Wikidata...")
komet_provenance = kh.search_komet_provenance_wikidata()

print(f"\nItems referencing KOMET/OPTIMETA as source: {len(komet_provenance)}")

if komet_provenance:
    for item in komet_provenance:
        print(f"  - {item.get('itemLabel', {}).get('value', 'N/A')}")
else:
    print("  (None found - this is expected at this stage)")

# Log to timeline
kh.add_observation(
    timeline,
    "wikidata_komet_provenance_count",
    len(komet_provenance),
    "wikidata",
    "Items with KOMET/OPTIMETA in P1343 source"
)

Searching for KOMET/OPTIMETA provenance markers in Wikidata...


Query timed out after 60s

Items referencing KOMET/OPTIMETA as source: 0
  (None found - this is expected at this stage)


True

### 1.2 P2860 (Cites Work) - Baseline for Partner Journals

We track **P2860 (cites work)** relationships for KOMET/OPTIMETA partner journals to establish a baseline. This allows us to measure the impact of collaboration over time:

- **Now**: Establish baseline counts before active metadata enrichment
- **Future**: Compare to measure growth in citation metadata

In [4]:
# Get all partner journals with Wikidata QIDs
partner_journals = kh.get_journals_with_wikidata()
print(f"Partner journals with Wikidata entries: {len(partner_journals)}")
print("\nJournals to track:")
for j in partner_journals:
    print(f"  - {j['name']} ({j['wikidata_qid']})")

Partner journals with Wikidata entries: 15

Journals to track:
  - Journal of South Asian Linguistics (Q122948152)
  - Free Neuropathology (Q108455809)
  - Jahrbuch für Christliche Sozialwissenschaften (Q1678617)
  - Journal für Kulturpflanzen (Q1455822)
  - VITIS - Journal of Grapevine Research (Q15756080)
  - Journal of Applied Botany and Food Quality (Q15764825)
  - Francia-Recensio (Q101247086)
  - Heidelberger Beiträge zum Finanz- und Steuerrecht (Q105103105)
  - Informationspraxis (Q46478422)
  - International Journal of Dream Research (Q96332444)
  - Journal of Dynamic Decision Making (Q50817185)
  - Archäologischer Anzeiger (Q636752)
  - Journal of Spatial Information Science (Q50814880)
  - Cognitio (Q111049844)
  - itdb - inter- und transdisziplinäre Bildung (Q107074231)


In [5]:
# Query current Wikidata statistics for each partner journal
import time

print("Querying Wikidata for partner journal statistics...")
print("(This may take a while due to rate limiting)\n")

journal_stats = []

for journal in partner_journals:
    qid = journal["wikidata_qid"]
    name = journal["name"]
    
    print(f"Querying {name}...", end=" ")
    
    # Get article count
    article_count = kh.count_journal_articles_wikidata(qid)
    
    # Get citation count (articles FROM this journal that cite other works)
    citation_count = kh.count_journal_citations_p2860(qid)
    
    stats = {
        "name": name,
        "partner": journal.get("partner", "Unknown"),
        "qid": qid,
        "wikidata_articles": article_count,
        "wikidata_citations_p2860": citation_count,
        "baseline_2022": journal.get("baseline_2022", {})
    }
    journal_stats.append(stats)
    
    print(f"articles={article_count}, citations={citation_count}")
    
    # Log to timeline
    kh.add_observation(
        timeline,
        f"wikidata_journal_{qid}_articles",
        article_count,
        "wikidata"
    )
    kh.add_observation(
        timeline,
        f"wikidata_journal_{qid}_citations_p2860",
        citation_count,
        "wikidata"
    )
    
    time.sleep(1)  # Rate limiting

Querying Wikidata for partner journal statistics...
(This may take a while due to rate limiting)

Querying Journal of South Asian Linguistics... 

articles=0, citations=0


Querying Free Neuropathology... 

articles=0, citations=0


Querying Jahrbuch für Christliche Sozialwissenschaften... 

...

articles=0, citations=0


Querying Cognitio... 

articles=0, citations=0


Querying itdb - inter- und transdisziplinäre Bildung... 

articles=14, citations=0


In [6]:
# Display journal statistics table
print("\n" + "="*80)
print("PARTNER JOURNAL WIKIDATA STATISTICS - BASELINE")
print("="*80)
print(f"{'Journal':<45} {'QID':<12} {'Articles':<10} {'Citations':<10}")
print("-"*80)

total_articles = 0
total_citations = 0

for s in journal_stats:
    articles = s['wikidata_articles'] or 0
    citations = s['wikidata_citations_p2860'] or 0
    total_articles += articles
    total_citations += citations
    
    print(f"{s['name'][:44]:<45} {s['qid']:<12} {articles:<10} {citations:<10}")

print("-"*80)
print(f"{'TOTAL':<45} {'':<12} {total_articles:<10} {total_citations:<10}")
print("\nNote: 'Citations' = outgoing P2860 relationships (articles citing other works)")
print("Low values are expected - this establishes the baseline before KOMET collaboration.")


PARTNER JOURNAL WIKIDATA STATISTICS - BASELINE
Journal                                       QID          Articles   Citations 
--------------------------------------------------------------------------------
Journal of South Asian Linguistics            Q122948152   0          0         
Free Neuropathology                           Q108455809   0          0         
Jahrbuch für Christliche Sozialwissenschafte  Q1678617     0          0         
Journal für Kulturpflanzen                    Q1455822     0          0         
VITIS - Journal of Grapevine Research         Q15756080    0          0         
Journal of Applied Botany and Food Quality    Q15764825    0          0         
Francia-Recensio                              Q101247086   0          0         
Heidelberger Beiträge zum Finanz- und Steuer  Q105103105   0          0         
Informationspraxis                            Q46478422    28         2         
International Journal of Dream Research       Q96332444    0 

### 1.3 Comparison with 2022 Baseline

The 2022 baseline data (from "Journals der Partner" document) allows us to track changes over time.

In [7]:
# Compare current stats with 2022 baseline
print("\n" + "="*90)
print("COMPARISON: 2022 BASELINE vs CURRENT")
print("="*90)
print(f"{'Journal':<35} {'2022 Art.':<10} {'Now Art.':<10} {'Change':<10} {'2022 Cit.':<10} {'Now Cit.':<10}")
print("-"*90)

for s in journal_stats:
    baseline = s.get('baseline_2022', {})
    baseline_art = baseline.get('wikidata_articles', 'N/A')
    current_art = s['wikidata_articles'] or 0
    baseline_cit = baseline.get('wikidata_with_citations', 'N/A')
    current_cit = s['wikidata_citations_p2860'] or 0
    
    # Calculate change
    if isinstance(baseline_art, int):
        change = current_art - baseline_art
        change_str = f"+{change}" if change >= 0 else str(change)
    else:
        change_str = "N/A"
    
    print(f"{s['name'][:34]:<35} {str(baseline_art):<10} {current_art:<10} {change_str:<10} {str(baseline_cit):<10} {current_cit:<10}")


COMPARISON: 2022 BASELINE vs CURRENT
Journal                             2022 Art.  Now Art.   Change     2022 Cit.  Now Cit.  
------------------------------------------------------------------------------------------
Journal of South Asian Linguistics  0          0          +0         0          0         
Free Neuropathology                 0          0          +0         0          0         
Jahrbuch für Christliche Sozialwis  0          0          +0         0          0         
Journal für Kulturpflanzen          0          0          +0         0          0         
VITIS - Journal of Grapevine Resea  0          0          +0         0          0         
Journal of Applied Botany and Food  3          0          -3         0          0         
Francia-Recensio                    1          0          -1         0          0         
Heidelberger Beiträge zum Finanz-   0          0          +0         0          0         
Informationspraxis                  55         28   

---

## 2. OpenCitations Crowdsourcing Analysis

**OpenCitations is the primary target platform** for KOMET citation metadata contributions.

The crowdsourcing repository (https://github.com/opencitations/crowdsourcing) allows trusted agents to contribute citation metadata via GitHub Issues.

### 2.1 Fetch All Issues

In [8]:
# Fetch all issues from OpenCitations crowdsourcing repo
github_token = os.environ.get('GITHUB_TOKEN')

print("Fetching issues from opencitations/crowdsourcing...")
all_issues = kh.get_all_opencitations_issues(token=github_token)
print(f"Total issues fetched: {len(all_issues)}")

# Log to timeline
kh.add_observation(
    timeline,
    "opencitations_total_issues",
    len(all_issues),
    "opencitations"
)

Fetching issues from opencitations/crowdsourcing...


Total issues fetched: 16


True

In [9]:
# Generate summary statistics
summary = kh.summarize_opencitations_contributions(all_issues)

print("\n" + "="*50)
print("OPENCITATIONS CROWDSOURCING SUMMARY")
print("="*50)
print(f"\nTotal submissions: {summary['total_issues']}")

print(f"\nBy Status:")
for status, count in sorted(summary['status_breakdown'].items(), key=lambda x: -x[1]):
    print(f"  {status}: {count}")

print(f"\nBy Creator:")
for creator, count in sorted(summary['by_creator'].items(), key=lambda x: -x[1]):
    komet_marker = " [KOMET]" if creator in kh.KOMET_CONTRIBUTORS else ""
    print(f"  {creator}: {count}{komet_marker}")


OPENCITATIONS CROWDSOURCING SUMMARY

Total submissions: 15

By Status:
  invalid: 11
  unknown: 2
  done: 1
  to_be_processed: 1

By Creator:
  arcangelo7: 11
  GaziYucel: 4 [KOMET]


### 2.2 KOMET Contributions Filter

Filter issues to show only those from known KOMET team contributors.

In [10]:
# Filter for KOMET contributions
print(f"Known KOMET contributors: {kh.KOMET_CONTRIBUTORS}")

komet_issues = kh.filter_komet_contributions(all_issues)
print(f"\nIssues from KOMET contributors: {len(komet_issues)}")

# Log to timeline
kh.add_observation(
    timeline,
    "opencitations_komet_issues",
    len(komet_issues),
    "opencitations"
)

if komet_issues:
    print("\nKOMET Contributions:")
    print("-" * 70)
    for issue in komet_issues:
        parsed = kh.parse_opencitations_issue(issue)
        title_short = parsed['title'][:55] + "..." if len(parsed['title']) > 55 else parsed['title']
        print(f"#{parsed['issue_number']}: {title_short}")
        print(f"  Status: {parsed['status']} | Created: {parsed['created_at'][:10]}")
        print(f"  URL: {parsed['url']}\n")
else:
    print("\n(No KOMET contributions found yet - this is the baseline)")

Known KOMET contributors: ['GaziYucel']

Issues from KOMET contributors: 5

KOMET Contributions:
----------------------------------------------------------------------
#6: deposit localhost doi:10.1234/37fs2v66
  Status: invalid | Created: 2025-10-10
  URL: https://github.com/opencitations/crowdsourcing/issues/6

#5: deposit localhost doi:10.1234/37fs2v66
  Status: invalid | Created: 2025-10-09
  URL: https://github.com/opencitations/crowdsourcing/issues/5

#4: deposit localhost doi:10.1234/37fs2v66
  Status: invalid | Created: 2025-10-09
  URL: https://github.com/opencitations/crowdsourcing/issues/4

#3: deposit localhost doi:10.0000/1rjphk47
  Status: invalid | Created: 2025-03-27
  URL: https://github.com/opencitations/crowdsourcing/issues/3

#2: Add GaziYucel to whitelist.txt
  Status: rejected | Created: 2023-04-04
  URL: https://github.com/opencitations/crowdsourcing/pull/2



In [11]:
# Calculate KOMET-specific metrics
komet_summary = kh.summarize_opencitations_contributions(komet_issues) if komet_issues else {'total_issues': 0, 'status_breakdown': {}}

successful = komet_summary['status_breakdown'].get('done', 0)
pending = komet_summary['status_breakdown'].get('to_be_processed', 0)
invalid = komet_summary['status_breakdown'].get('invalid', 0)

print("\n" + "="*50)
print("KOMET OPENCITATIONS METRICS")
print("="*50)
print(f"\nTotal submissions: {komet_summary['total_issues']}")
print(f"Successfully processed (done): {successful}")
print(f"Pending processing: {pending}")
print(f"Invalid/rejected: {invalid}")

if komet_summary['total_issues'] > 0:
    success_rate = (successful / komet_summary['total_issues']) * 100
    print(f"\nSuccess rate: {success_rate:.1f}%")

# Log detailed stats to timeline
kh.add_observation(timeline, "opencitations_komet_done", successful, "opencitations")
kh.add_observation(timeline, "opencitations_komet_pending", pending, "opencitations")
kh.add_observation(timeline, "opencitations_komet_invalid", invalid, "opencitations")


KOMET OPENCITATIONS METRICS

Total submissions: 4
Successfully processed (done): 0
Pending processing: 0
Invalid/rejected: 4

Success rate: 0.0%


True

---

## 3. Partner Journal Discovery & Status

This section provides an overview of all KOMET/OPTIMETA collaboration partners and their current status in Wikidata and OpenCitations.

In [12]:
# Display all partner journals
all_journals = kh.COLLABORATION_PARTNERS["journals"]
journals_without_wd = kh.COLLABORATION_PARTNERS.get("journals_without_wikidata", [])

print("="*80)
print("KOMET/OPTIMETA COLLABORATION PARTNERS - JOURNAL STATUS")
print("="*80)

print(f"\nJournals WITH Wikidata entries: {len(all_journals)}")
print(f"Journals WITHOUT Wikidata entries: {len(journals_without_wd)}")

print("\n--- Journals with Wikidata QIDs ---")
for j in all_journals:
    print(f"  [{j['wikidata_qid']}] {j['name']}")
    print(f"      Partner: {j.get('partner', 'N/A')} | URL: {j.get('url', 'N/A')}")

print("\n--- Journals without Wikidata entries (potential for expansion) ---")
for j in journals_without_wd:
    print(f"  - {j['name']} ({j.get('partner', 'N/A')})")

KOMET/OPTIMETA COLLABORATION PARTNERS - JOURNAL STATUS

Journals WITH Wikidata entries: 15
Journals WITHOUT Wikidata entries: 7

--- Journals with Wikidata QIDs ---
  [Q122948152] Journal of South Asian Linguistics
      Partner: KIM Universität Konstanz | URL: https://ojs.ub.uni-konstanz.de/jsal/
  [Q108455809] Free Neuropathology
      Partner: WWU Münster | URL: https://www.uni-muenster.de/Ejournals/index.php/fnp
  [Q1678617] Jahrbuch für Christliche Sozialwissenschaften
      Partner: WWU Münster | URL: https://www.uni-muenster.de/Ejournals/index.php/jcsw
  [Q1455822] Journal für Kulturpflanzen
      Partner: Julius Kühn-Institut | URL: https://ojs.openagrar.de/index.php/Kulturpflanzenjournal
  [Q15756080] VITIS - Journal of Grapevine Research
      Partner: Julius Kühn-Institut | URL: https://ojs.openagrar.de/index.php/VITIS
  [Q15764825] Journal of Applied Botany and Food Quality
      Partner: Julius Kühn-Institut | URL: https://ojs.openagrar.de/index.php/JABFQ
  [Q101247086] Fr

In [13]:
# Display platforms
print("\n--- Publishing Platforms ---")
for p in kh.COLLABORATION_PARTNERS["platforms"]:
    print(f"  - {p['name']}")
    print(f"    {p['url']}")

print("\n--- Infrastructure Partners ---")
for i in kh.COLLABORATION_PARTNERS["infrastructure"]:
    notes = f" - {i['notes']}" if i.get('notes') else ""
    print(f"  - {i['name']}{notes}")


--- Publishing Platforms ---
  - KIM - Universität Konstanz
    https://www.kim.uni-konstanz.de/
  - WWU E-Journals Münster
    https://www.uni-muenster.de/Ejournals/
  - e-journals Julius Kühn-Institut
    https://ojs.openagrar.de
  - heiJOURNALS Heidelberg
    https://journals.ub.uni-heidelberg.de
  - TIB Open Publishing
    https://www.tib-op.org/
  - ZHB Luzern
    https://www.zhbluzern.ch/

--- Infrastructure Partners ---
  - Public Knowledge Project (PKP)
  - OpenCitations - Primary target platform


---

## 4. Summary & Export

### 4.1 Key Findings

In [14]:
# Compile report data
report_data = {
    "report_timestamp": kh.format_timestamp(),
    "platform_priority": {
        "primary": "OpenCitations",
        "secondary": "Wikidata (monitoring only)",
        "planned": "OpenAIRE"
    },
    "wikidata": {
        "p1343_scholarly_count": p1343_count,
        "p1343_limitation_note": "P1343 rarely used for scholarly articles (~14 items); not suitable for tracking",
        "komet_provenance_items": len(komet_provenance) if komet_provenance else 0,
        "partner_journals_tracked": len(partner_journals),
        "partner_journals_total_articles": total_articles,
        "partner_journals_total_citations_p2860": total_citations
    },
    "opencitations": {
        "total_crowdsourcing_issues": summary['total_issues'],
        "komet_contributions": komet_summary['total_issues'],
        "komet_successful": successful,
        "komet_pending": pending,
        "komet_invalid": invalid,
        "komet_contributors": kh.KOMET_CONTRIBUTORS
    },
    "partner_journals": journal_stats
}

print("Report Summary:")
print(json.dumps(report_data, indent=2, default=str))

Report Summary:
{
  "report_timestamp": "2026-01-22T11:32:25.290329Z",
  "platform_priority": {
    "primary": "OpenCitations",
    "secondary": "Wikidata (monitoring only)",
    "planned": "OpenAIRE"
  },
  "wikidata": {
    "p1343_scholarly_count": 14,
    "p1343_limitation_note": "P1343 rarely used for scholarly articles (~14 items); not suitable for tracking",
    "komet_provenance_items": 0,
    "partner_journals_tracked": 15,
    "partner_journals_total_articles": 42,
    "partner_journals_total_citations_p2860": 2
  },
  "opencitations": {
    "total_crowdsourcing_issues": 15,
    "komet_contributions": 4,
    "komet_successful": 0,
    "komet_pending": 0,
    "komet_invalid": 4,
    "komet_contributors": [
      "GaziYucel"
    ]
  },
  "partner_journals": [
    {
      "name": "Journal of South Asian Linguistics",
      "partner": "KIM Universit\u00e4t Konstanz",
      "qid": "Q122948152",
      "wikidata_articles": 0,
      "wikidata_citations_p2860": 0,
      "baseline_2022"

### 4.2 Save Timeline & Export Report

In [15]:
# Save timeline with all observations
kh.save_timeline(timeline)
print(f"Timeline saved to: {kh.TIMELINE_LOG_FILE}")
print(f"Total observations in timeline: {len(timeline['observations'])}")

# Save current report snapshot
report_file = "komet_report_data.json"
kh.save_json(report_data, report_file)
print(f"Report data saved to: {report_file}")

Timeline saved to: komet_timeline.json
Total observations in timeline: 37
Report data saved to: komet_report_data.json


In [16]:
# Display timeline summary
print("\n" + "="*60)
print("TIMELINE SUMMARY - Latest Values")
print("="*60)

latest = kh.get_latest_metrics(timeline)
for metric, obs in sorted(latest.items()):
    print(f"{metric}: {obs['value']} (as of {obs['timestamp'][:10]})")


TIMELINE SUMMARY - Latest Values
opencitations_komet_done: 0 (as of 2026-01-22)
opencitations_komet_invalid: 4 (as of 2026-01-22)
opencitations_komet_issues: 5 (as of 2026-01-22)
opencitations_komet_pending: 0 (as of 2026-01-22)
opencitations_total_issues: 16 (as of 2026-01-22)
wikidata_journal_Q101247086_articles: 0 (as of 2026-01-22)
wikidata_journal_Q101247086_citations_p2860: 0 (as of 2026-01-22)
wikidata_journal_Q105103105_articles: 0 (as of 2026-01-22)
wikidata_journal_Q105103105_citations_p2860: 0 (as of 2026-01-22)
wikidata_journal_Q107074231_articles: 14 (as of 2026-01-22)
wikidata_journal_Q107074231_citations_p2860: 0 (as of 2026-01-22)
wikidata_journal_Q108455809_articles: 0 (as of 2026-01-22)
wikidata_journal_Q108455809_citations_p2860: 0 (as of 2026-01-22)
wikidata_journal_Q111049844_articles: 0 (as of 2026-01-22)
wikidata_journal_Q111049844_citations_p2860: 0 (as of 2026-01-22)
wikidata_journal_Q122948152_articles: 0 (as of 2026-01-22)
wikidata_journal_Q122948152_citatio

---

## 5. Next Steps

Based on this analysis:

### Immediate
1. **OpenCitations focus**: Monitor GaziYucel contributions and encourage more submissions
2. **Baseline established**: Current Wikidata statistics captured for partner journals
3. **Timeline logging**: Re-run this notebook periodically to track changes

### Future Work
- [ ] Explore OpenAIRE API for additional citation data
- [ ] Add more KOMET team members to contributor filter
- [ ] Set up weekly CI automation
- [ ] Create visualization dashboards for trends
- [ ] Add Wikidata entries for journals without QIDs

---

## Funding

This work is funded by the **German Federal Ministry of Education and Research (BMBF)** under grant number **16TOA039**.

The KOMET project is part of the funding initiative "Förderung von Projekten zur Etablierung einer gelebten Open-Access-Kultur in der deutschen Forschungs- und Wissenschaftspraxis" (Funding for projects to establish a lived Open Access culture in German research and scientific practice).

![BMBF Logo](https://www.bmbf.de/SiteGlobals/Frontend/Images/icons/logo.svg?__blob=normal)

---

## License & Data Sources

### This Notebook

- **Code**: [CC0 1.0 Universal](https://creativecommons.org/publicdomain/zero/1.0/)
- **Report outputs**: [CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/)

### External Data Sources

This analysis uses data from the following sources, each with their own licensing terms:

| Source | License | Terms |
|--------|---------|-------|
| **Wikidata** | CC0 1.0 | [Wikidata Licensing](https://www.wikidata.org/wiki/Wikidata:Licensing) |
| **OpenCitations** | CC0 1.0 | [OpenCitations Licensing](https://opencitations.net/about#licensing) |
| **GitHub API** | GitHub ToS | [GitHub Terms of Service](https://docs.github.com/en/site-policy/github-terms) |

### Citation

If you use this work, please cite:

```bibtex
@software{komet_report,
  author = {KOMET Project Team},
  title = {KOMET Evaluation Report: Tracking Open Metadata Contributions},
  url = {https://github.com/TIBHannover/komet-report},
  year = {2024-2025}
}
```

**Project Website**: https://projects.tib.eu/komet